In [2]:
import numpy as np
import utils
import planners
from models import CostFunction
import constants as c
import matplotlib.pyplot as plt
import time
import trajoptpy.math_utils as mu
import random
import itertools
import pickle
%matplotlib inline

In [3]:
env, robot = utils.setup()

In [4]:
files = {
    'happy': 'allan_happy_rotation_1',
    'sad': 'allan_sad_2',
    'hesitant': 'allan_hesitant_rotation_1'
}
style = 'hesitant'

In [4]:
cf = CostFunction(robot, num_dofs=7, normalize=True)
cf.load_model('./saves/experiments/{:s}/'.format(files[style]))
custom_cost = {'NN': planners.get_trajopt_error_cost(cf)}

INFO:tensorflow:Restoring parameters from ./saves/experiments/allan_hesitant_rotation_1/


In [5]:
with open('./data/experiments/{:s}.pkl'.format(files[style]), 'rb') as f:
    sess_vars = pickle.load(f)

## Testing: 1, 4, 5

In [12]:
start_idx, end_idx = c.sg_test_idcs[5]
start = c.configs[start_idx]
end = c.configs[end_idx]

In [13]:
robot.SetActiveDOFValues(start)
with env:
    res0 = planners.trajopt_simple_plan(
        env, robot, end,
        custom_traj_costs=custom_cost,
        joint_vel_coeff=sess_vars['joint_vel_coeff'])
    traj0 = utils.waypoints_to_traj(
        env, robot, res0.GetTraj(), sess_vars['speed'], None)

In [14]:
np.save('./exp_trajs/handover_hesitant_nn', res0.GetTraj())

In [7]:
robot.SetActiveDOFValues(start)

In [19]:
robot.ExecuteTrajectory(traj0)

RaveCreateInterface(RaveGetEnvironment(1),InterfaceType.trajectory,'GenericTrajectory')

## Synthetic costs

In [6]:
def sad_cost_height(x):
    val = 1. * feature_height(robot, x)
    return val
def sad_cost_orientation(x):
    return .1 * feature_orientation(robot, x)

synthetic_sad_costs = {
    'height': sad_cost_height,
    'orientation': sad_cost_orientation
}

def happy_cost_height(x):
    val = 5 * feature_height2(robot, x)
    return val
def happy_cost_orientation(x):
    return -10 * feature_orientation(robot, x)

synthetic_happy_costs = {
    'height': happy_cost_height,
    'orientation': happy_cost_orientation
}

def hesitation_traj_cost(x):
    x = x.reshape((10, 7))
    return .4*np.linalg.norm(x[3] - x[2]) + .2 * np.linalg.norm(x[2] - x[1]) + .2 * np.linalg.norm(x[4] - x[2])

synthetic_hesitation_costs = {
    'hesitation': hesitation_traj_cost
}

In [25]:
robot.SetActiveDOFValues(start)
with env:
    res2 = planners.trajopt_simple_plan(
        env, robot, end,
        custom_costs=synthetic_sad_costs)
    traj2 = utils.waypoints_to_traj(
        env, robot, res2.GetTraj(), sess_vars['speed'], None)

In [27]:
robot.SetActiveDOFValues(start)
with env:
    res2 = planners.trajopt_simple_plan(
        env, robot, end,
        custom_costs=synthetic_happy_costs)
    traj2 = utils.waypoints_to_traj(
        env, robot, res2.GetTraj(), sess_vars['speed'], None)

In [29]:
robot.SetActiveDOFValues(start)
with env:
    res2 = planners.trajopt_simple_plan(
        env, robot, end,
        custom_traj_costs=synthetic_hesitation_costs)
    traj2 = utils.waypoints_to_traj(
        env, robot, res2.GetTraj(), sess_vars['speed'], None)

In [53]:
robot.SetActiveDOFValues(start)

In [54]:
robot.ExecuteTrajectory(traj2)

RaveCreateInterface(RaveGetEnvironment(1),InterfaceType.trajectory,'GenericTrajectory')

## Default

In [27]:
robot.SetActiveDOFValues(start)
with env:
    res1 = planners.trajopt_simple_plan(
        env, robot, end)
    traj1 = utils.waypoints_to_traj(
        env, robot, res1.GetTraj(), 0.5, None)

In [8]:
robot.SetActiveDOFValues(start)

In [17]:
robot.ExecuteTrajectory(traj1)

RaveCreateInterface(RaveGetEnvironment(1),InterfaceType.trajectory,'GenericTrajectory')